In [36]:
import requests
import pandas as pd

url_prefix = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-evaluation/'
docs_url = url_prefix + 'search_evaluation/documents-with-ids.json'
documents = requests.get(docs_url).json()

ground_truth_url = url_prefix + 'search_evaluation/ground-truth-data.csv'
df_ground_truth = pd.read_csv(ground_truth_url)
ground_truth = df_ground_truth.to_dict(orient='records')

In [31]:
from tqdm.auto import tqdm

def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q['question'])
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total)
    }

# Q1. Minsearch text

In [3]:
import minsearch

In [37]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [4]:
index = minsearch.Index(
    text_fields=["question", "section", "text"],
    keyword_fields=["course", "id"]
)

In [5]:
index.fit(documents)

In [39]:
def search(query):
    boost = {'question': 1.5, 'section': 0.1}

    results = index.search(query=query,
             num_results=5,
             boost_dict=boost)

    return results

In [41]:
#search(documents[0]['question'])

In [40]:
evaluate(ground_truth, search)

  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.8013831856494489, 'mrr': 0.6818312801671366}

# Q2. Vector search for question (embedding)

In [33]:
from minsearch import VectorSearch

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

In [38]:
texts = []

for doc in documents:
    t = doc['question']
    texts.append(t)

pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)
X = pipeline.fit_transform(texts)

In [42]:
vindex = VectorSearch(keyword_fields={'course'})
vindex.fit(X, documents)

In [44]:
def vector_search(query):
    boost = {'question': 1.5, 'section': 0.1}

    results = vindex.search(query=query,
             num_results=5,
             boost_dict=boost)

    return results

In [45]:
evaluate(ground_truth, vector_search)

  0%|          | 0/4627 [00:00<?, ?it/s]

TypeError: VectorSearch.search() got an unexpected keyword argument 'query'